In [ ]:
from flask import render_template
from flask import Flask, jsonify, request
from flask_cors import CORS


from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
from urllib.request import urlopen
import json
import sys
sys.path.append("../")
from meta.parser import getWords

#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./npm/chromedriver',options=options)

#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

def scrape_page(dir_url,browser):
    l = []
    #execute js on webpage to load listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,browser)
    try:
        pv4 =  soup.find('div',class_='pv4')
        for a in pv4.find_all('a'):
            try:
                lib = a.contents[0] #get url
                l.append(lib)
            except:
                pass
    except:
        try:
            listp10 = get_js_soup(dir_url,browser).find('ul',class_='list p10')
            for a in listp10.find_all('a'):
                try:
                    lib = a.contents[0] #get url
                    l.append(lib)
                except:
                    pass
        except:
            pass
    return l

def fetch(lib):
    dir_url = "https://www.npmjs.com/package/"+str(lib) #base url
    return scrape_page(dir_url,browser)       

def test1(inp):
    listTopics = []
    for i in inp:
        results = fetch(i)
        for j in results:
            if j not in listTopics:
                listTopics.append(j)
    with open('../topic_lib_dic.json', 'r') as file:
        topic_lib_dic = json.loads(file.read())
    file.close()
    ret = {}
    for i in listTopics:
        if i in topic_lib_dic.keys():
            for j in topic_lib_dic[i]:
                if j not in ret.keys():
                    ret[j] = 1
                else:
                    ret[j] += 1
    return sorted(ret.items(), key=lambda kv: kv[1])[::-1]

app = Flask(__name__)
CORS(app)
app.config['SECRET_KEY'] = 'any secret string'
# app.config["DEBUG"] = True

@app.route('/', methods=['GET', 'POST'])

def index():
    with open("../scraper/data.txt") as file:
        lib_list = [line[1:-2].split(",") for line in file.readlines()]
    return "jaja"
    

@app.route('/test', methods=['GET'])
def test():
    query = request.args.get('data').replace(","," ")
    print("query is: ",query)
    words = getWords(query, 1, 20)
    return jsonify(words)


@app.route('/topic', methods=['GET'])
def topic():
    args = request.args.get('data').split(",")
    print(args)
    return jsonify(args)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/May/2019 17:19:33] "OPTIONS /test?data=axios,gh-pages,node-sass,prop-types,react,react-dom,react-router-dom,react-scripts,semantic-ui-css,semantic-ui-react HTTP/1.1" 200 -
[2019-05-07 17:19:33,535] ERROR in app: Exception on /test [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask_cors\extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_com

query is:  axios gh-pages node-sass prop-types react react-dom react-router-dom react-scripts semantic-ui-css semantic-ui-react
